In [1]:
import logging
import pickle

from pathlib import Path
from random import choice

from geoloc_imc_2023.cbg import CBG, get_prefix_from_ip
from geoloc_imc_2023.default import (
    ANCHOR_PATH
)

logging.basicConfig(level=logging.INFO)

NB_TARGET = 10
NB_VP = 10

ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}

# load hitlist
hitlist_file = Path(".") / "../datasets/parsed_hitlist.pickle"
with open(hitlist_file, "rb") as f:
    hitlist_addr = pickle.load(f)

# load anchors
anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

targets= [choice(list(anchors)) for _ in range(0, NB_TARGET)]
vps = list(set(anchors).difference(set(targets)))[:NB_VP]

logging.info(f"nb targets: {len(targets)} : {[target for target in targets]}")
logging.info(f"nb_vps : {len(vps)} {[vp for vp in vps]}")

cbg = CBG(targets, vps, ripe_credentials, hitlist_addr, anchors)

# get target prefixes
target_prefixes = []
for target_addr in targets:
    target_prefix = get_prefix_from_ip(target_addr)
    target_prefixes.append(target_prefix)

# initialization: start measurements from all vps to targets
init_measurement_ids = cbg.start_init_measurements(target_prefixes)

# retreive measurement results from RIPE Atlas
logging.info(init_measurement_ids)

INFO:root:nb targets: 10 : ['37.10.41.14', '103.196.37.98', '178.251.24.49', '207.246.74.246', '103.122.190.141', '192.42.42.166', '152.195.206.35', '37.252.251.26', '188.172.218.22', '63.222.190.5']
INFO:root:nb_vps : 10 ['34.97.153.243', '212.9.170.190', '107.162.219.5', '45.135.151.45', '176.74.17.75', '200.61.6.78', '212.72.227.133', '216.37.47.154', '68.64.60.178', '94.203.76.219']
INFO:root:starting measurement for target_prefix='37.10.41.0' with ['37.10.41.1', '37.10.41.2', '37.10.41.3'] with 10 self.anchors
INFO:root:starting measurement for target_prefix='103.196.37.0' with ['103.196.37.1', '103.196.37.2', '103.196.37.10'] with 10 self.anchors
INFO:root:starting measurement for target_prefix='178.251.24.0' with ['178.251.24.1', '178.251.24.2', '178.251.24.3'] with 10 self.anchors
INFO:root:starting measurement for target_prefix='207.246.74.0' with ['207.246.74.4', '207.246.74.7', '207.246.74.9'] with 10 self.anchors
INFO:root:starting measurement for target_prefix='103.122.190

In [2]:
import pickle

# save results
out_file = f"../results/init_measurement_ids.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(init_measurement_ids,f)

../results/init_measurement_ids.pickle


## get init results

In [1]:
import pickle
from pathlib import Path

from geoloc_imc_2023.query_api import retreive_single_measurement

measurement_ids_file = Path(".") / "../results/init_measurement_ids.pickle"
with open(measurement_ids_file, "rb") as f:
    init_measurement_ids = pickle.load(f)

# load anchors
anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}

measurement_results = {}
for prefix in init_measurement_ids:
    for measurement_id in init_measurement_ids[prefix]:
        measurement_results[prefix] = retreive_single_measurement(measurement_id, anchors, ripe_credentials["key"])

In [2]:
import pickle

# save results
out_file = f"../results/init_measurement.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurement_results,f)

../results/init_measurement.pickle


## SECOND PART: get ip geoloc

## TODO

1. get target ips (3) for all anchors
2. make ping from all anchors to all targets
3. select a set of sufficiently diversified probes (Metis) or simple

4. (probe validation) why not taking all achons and issuing pings towards the rest of probes? then cbg -> depending on results -> check

## TESTING

In [4]:
import pickle
from pathlib import Path

in_file = Path(".") / "../datasets/parsed_hitlist.pickle"
with open(in_file, "rb") as f:
    ip_hitlist = pickle.load(f)

anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)


In [41]:
from random import randint
from ipaddress import IPv4Address, IPv4Network
from geoloc_imc_2023.atlas_probing import RIPEAtlas
from collections import defaultdict
from datetime import datetime

RIPE_ACCOUNT = "timur.friedman@sorbonne-universite.fr"
RIPE_KEY = "b3d3d4fc-724e-4505-befe-1ad16a70dc87"
MAX_NUMBER_OF_VPS = 100
NB_PACKETS = 3

def get_target_hitlist(anchor_ip: str, hitlist_size: int = 3) -> list[str]:
    """from ip, return a list of target ips"""
    prefix_anchor = anchor_ip.split(".")[:-1]
    prefix_anchor.append("0")
    prefix_anchor = ".".join(prefix_anchor)
    ip_list = []
    try:
        ip_list = ip_hitlist[prefix_anchor]
    except KeyError:
        pass

    if len(ip_list) < hitlist_size:
        ip = anchor_ip.split(".")[:-1]
        ip.extend('0')
        ip = ".".join(ip)
        prefix_target = IPv4Network(ip + '/' + str(24))

        ip_list.extend([str(prefix_target[randint(1,254)]) for _ in range(0,hitlist_size - len(ip_list))])
    if len(ip_list) > hitlist_size:
        ip_list = ip_list[:hitlist_size]
    return ip_list


driver = RIPEAtlas(RIPE_ACCOUNT, RIPE_KEY)
measurements = defaultdict(dict)
dry_run = False
for i , ip_target in enumerate(anchors):

    if i > 10: break

    # get ip_target_list
    random_target_list = get_target_hitlist(ip_target, 3)
    # add the real ip_target to verify that geoloc prefixes is valid assumption
    random_target_list.append(ip_target)

    # get vp ids
    vp_ids = [anchors[ip_vp]['id'] for ip_vp in anchors if ip_vp != ip_target]
    assert len(vp_ids) == len(anchors) - 1
    # select a number of vps (testing)
    vp_ids = vp_ids[:MAX_NUMBER_OF_VPS]

    print(f"starting measurement for {ip_target=} with {[ip for ip in random_target_list]} with {len(vp_ids)} anchors")    

    ip_target_measurement_ids = []
    for ip in random_target_list:
        if dry_run:
            print(f"measurement for {ip}")
            continue
        else:
            # TODO: parralelize post requests otherwise it takes too much time
            # probe and retreive measurements
            measurement_id = driver.probe(str(ip), vp_ids, NB_PACKETS)
            try:
                measurements[ip_target]["id"].append(measurement_id)
            except KeyError:
                measurements[ip_target]["id"] = [measurement_id]
        

starting measurement for ip_target='213.225.160.239' with ['213.225.160.14', '213.225.160.140', '213.225.160.159', '213.225.160.239'] with 100 anchors
starting measurement for ip_target='145.220.0.55' with ['145.220.0.1', '145.220.0.28', '145.220.0.29', '145.220.0.55'] with 100 anchors
starting measurement for ip_target='5.28.0.17' with ['5.28.0.6', '5.28.0.13', '5.28.0.14', '5.28.0.17'] with 100 anchors
starting measurement for ip_target='193.171.255.2' with ['193.171.255.1', '193.171.255.2', '193.171.255.9', '193.171.255.2'] with 100 anchors
starting measurement for ip_target='192.65.184.54' with ['192.65.184.42', '192.65.184.54', '192.65.184.69', '192.65.184.54'] with 100 anchors
starting measurement for ip_target='185.42.136.158' with ['185.42.136.1', '185.42.136.2', '185.42.136.3', '185.42.136.158'] with 100 anchors
starting measurement for ip_target='197.80.104.36' with ['197.80.104.33', '197.80.104.34', '197.80.104.35', '197.80.104.36'] with 100 anchors
starting measurement for 

In [42]:
for target_addr in measurements:
    print(f"target ip : {target_addr} : {[id for id in measurements[target_addr]['id']]}")

target ip : 213.225.160.239 : [52572968, 52572971, 52572973, 52572975]
target ip : 145.220.0.55 : [52572977, 52572980, 52572982, 52572984]
target ip : 5.28.0.17 : [52572986, 52572989, 52572991, 52572994]
target ip : 193.171.255.2 : [52572998, 52573001, 52573004, 52573006]
target ip : 192.65.184.54 : [52573008, 52573011, 52573013, 52573016]
target ip : 185.42.136.158 : [52573018, 52573020, 52573023, 52573025]
target ip : 197.80.104.36 : [52573027, 52573029, 52573032, 52573034]
target ip : 200.7.84.24 : [52573037, 52573039, 52573042, 52573045]
target ip : 76.26.115.194 : [52573047, 52573050, 52573052, 52573054]
target ip : 194.150.191.46 : [52573056, 52573058, 52573061, 52573063]
target ip : 199.10.66.231 : [52573065, 52573069, 52573071, 52573074]


In [43]:
import pickle

# save results
date = datetime.now()
out_file = f"../results/measurement_ids.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurements,f)

../results/measurement_ids.pickle


In [44]:
import requests
import time
from typing import List
import pickle
from collections import defaultdict

# save results
in_file = f"../results/measurement_ids.pickle"
with open(in_file, "rb") as f:
    measurements = pickle.load(f)

anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

print(f"retreive measurements for {len(measurements)} ip addresses")

key = "b3d3d4fc-724e-4505-befe-1ad16a70dc87"
for i, target_addr in enumerate(measurements):

    measurements[target_addr]["target_results"] = []
    measurements[target_addr]["prefix_results"] = defaultdict(list)

    for measurement_id in measurements[target_addr]["id"]:
        url = f"https://atlas.ripe.net/api/v2/measurements/{measurement_id}/results/?key={key}"

        timeout = 60
        for _ in range(timeout):

            response = requests.get(url, timeout=20).json()
            if response:
                break
            time.sleep(2)

        for result in response:
            # parse results and calculate geoloc
            if result.get('result') is not None:
                
                dst_addr = result['dst_addr']
                vp_ip = result['from']

                if type(result['result']) == list:
                    rtt_list = [list(rtt.values())[0] for rtt in result['result']]
                else:
                    rtt_list = [result['result']["rtt"]]

                # remove stars from results
                rtt_list = list(filter(lambda x: x != "*", rtt_list))
                if not rtt_list: 
                    continue
                
                # get min rtt
                min_rtt = min(rtt_list)

                if min_rtt == "*": continue

                # both vp and target coordinates
                vp_lat = anchors[vp_ip]['latitude']
                vp_lon = anchors[vp_ip]['longitude']

                if dst_addr == target_addr:
                    measurements[target_addr]["target_results"].append({
                        "node": vp_ip,
                        "min_rtt": min_rtt,
                        "rtt_list": rtt_list,
                        "vp_lat": vp_lat,
                        "vp_lon": vp_lon,
                    })
                else: 
                    measurements[target_addr]["prefix_results"][dst_addr].append({
                        "node": vp_ip,
                        "min_rtt": min_rtt,
                        "rtt_list": rtt_list,
                        "vp_lat": vp_lat,
                        "vp_lon": vp_lon,
                    })
            else:
                print(f"no results: {result}")
    
    # sort delay
    measurements[target_addr]["target_results"] = sorted(measurements[target_addr]["target_results"], key = lambda x: x["min_rtt"])

    for dst_addr in  measurements[target_addr]["prefix_results"]:

        try:
            measurements[target_addr]["prefix_results"][dst_addr] = sorted(
                measurements[target_addr]["prefix_results"][dst_addr], 
                key = lambda x: x["min_rtt"]
            )
        except TypeError:
            raise RuntimeError(measurements[target_addr]["prefix_results"][dst_addr])
    


retreive measurements for 11 ip addresses


In [45]:
import pickle

for i, ip_target in enumerate(measurements):
    if i > 10: break
    print(f"results for ip: {ip_target}:")
    print(f"len {len(measurements[ip_target]['target_results'])}")
    for result in measurements[ip_target]["target_results"]:
        print("ip result", result)
    for ip_dst in measurements[ip_target]["prefix_results"]:
        print("prefix results:",measurements[ip_target]["prefix_results"][ip_dst])
    
# save results
date = datetime.now()
out_file = f"../results/all_anchors_to_all_anchors_test.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurements,f)

results for ip: 213.225.160.239:
len 100
ip result {'node': '185.138.143.34', 'min_rtt': 4.026103, 'rtt_list': [4.271183, 4.177587, 4.026103], 'vp_lat': 50.1285, 'vp_lon': 8.5975}
ip result {'node': '217.198.242.227', 'min_rtt': 4.648829, 'rtt_list': [4.66626, 4.756188, 4.648829], 'vp_lat': 49.9905, 'vp_lon': 8.2385}
ip result {'node': '80.67.163.251', 'min_rtt': 6.353934, 'rtt_list': [6.396421, 6.353934, 6.572103], 'vp_lat': 48.8585, 'vp_lon': 2.3785}
ip result {'node': '132.227.123.3', 'min_rtt': 7.237546, 'rtt_list': [7.237546, 7.282237, 7.262402], 'vp_lat': 48.8515, 'vp_lon': 2.3605}
ip result {'node': '88.198.173.54', 'min_rtt': 7.400731, 'rtt_list': [7.400731, 7.604218, 7.43489], 'vp_lat': 49.4485, 'vp_lon': 11.0085}
ip result {'node': '213.9.97.164', 'min_rtt': 7.774361, 'rtt_list': [7.774361, 7.977515, 8.196444], 'vp_lat': 51.2285, 'vp_lon': 6.7675}
ip result {'node': '31.172.2.253', 'min_rtt': 9.003416, 'rtt_list': [9.006763, 9.036641, 9.003416], 'vp_lat': 51.5205, 'vp_lon': 7

## Apply CBG methodology

In [48]:
from pathlib import Path
import random
import pickle
from copy import copy

from geoloc_imc_2023.helpers import polygon_centroid, haversine, circle_intersections
from geoloc_imc_2023.draw_results import draw_results

def select_best_guess_centroid(target_addr, measurement_results):
    """
    Find the best guess
    that is the location of the vantage point closest to the centroid.
    """
    probe_circles = {}
    print(f"NUMBER OF MEASUREMENT RESULTS FOR TARGET IP: {target_addr} : {len(measurement_results)}")
    for _, result in enumerate(measurement_results):
        vp_ip = result["node"]
        lat = result["vp_lat"]
        lon = result["vp_lon"]
        min_rtt = result["min_rtt"]

        # too inflated RTT means that measurement will not provide usefull info
        if isinstance(min_rtt, float):
                probe_circles[vp_ip] = (
                    lat,
                    lon,
                    min_rtt,
                    None,
                    None,
                )
                print(f"vp_anchor = {vp_ip} with results: {min_rtt}")
    print()

    # draw circles
    if not probe_circles: return

    intersections = circle_intersections(probe_circles.values(), speed_threshold=4/9)
    print("intersecion:", intersections)
    target = (anchors[target_addr]["latitude"], anchors[target_addr]["longitude"])

    centroid = None
    if intersections:
        centroid = polygon_centroid(intersections)
        print("centroid:", centroid)
        target_description = anchors[target_addr]

        if centroid:
            for ip_src, circle in probe_circles.items():

                measured_d = haversine((circle[0], circle[1]), centroid)
                calculated_d = haversine((target_description["latitude"], target_description["longitude"]), centroid)

                print("ip_src:",ip_src,"dst (measured):", measured_d, "dst (calculated):", calculated_d)
    
    draw_results(copy(probe_circles), intersections,centroid, target)

    return centroid

in_file = Path(".") / "../results/all_anchors_to_all_anchors_test.pickle"
with open(in_file, "rb") as f:
    measurements = pickle.load(f)

anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

# select target and vp list for analyssi
# TARGET_SIZE = 5
# target_list = [random.choice(list(measurements)) for _ in range(0, TARGET_SIZE)]
# vp_list = set(measurements).difference(set(target_list))

target_list = list(measurements.keys())
vp_list = list(measurements.keys())

print(f"target size: {len(target_list)}")
print(f"vp size: {len(vp_list)}")

geoloc_results = {}
for i, target_addr in enumerate(target_list):
    if i > 10: break

    # get measurements for vp_dataset
    vp_results = []
    for vp_result in measurements[target_addr]['target_results']:
        if vp_result['node'] in vp_list:
            vp_results.append(vp_result)
    

    print("available measurements for ip:",len(vp_results))
    shortest_delay_results = vp_results[:10]

    centroid = select_best_guess_centroid(target_addr, shortest_delay_results)
    if centroid:
        geoloc_results[target_addr] = {
            "estimated_lat": centroid[0],
            "estimated_lon": centroid[1],
        }

geoloc_out_file = Path(".") / "../results/all_anchors_towards_all_anchors_geoloc.pickle"
with open(geoloc_out_file, "wb") as f:
    pickle.dump(geoloc_results, f)


target size: 11
vp size: 11
available measurements for ip: 10
NUMBER OF MEASUREMENT RESULTS FOR TARGET IP: 213.225.160.239 : 10
vp_anchor = 145.220.0.55 with results: 9.598958
vp_anchor = 199.10.66.231 with results: 10.563863
vp_anchor = 194.150.191.46 with results: 15.393551
vp_anchor = 193.171.255.2 with results: 18.839501
vp_anchor = 192.65.184.54 with results: 18.954774
vp_anchor = 5.28.0.17 with results: 28.570676
vp_anchor = 185.42.136.158 with results: 30.415617
vp_anchor = 76.26.115.194 with results: 86.112694
vp_anchor = 200.7.84.24 with results: 179.835779
vp_anchor = 197.80.104.36 with results: 189.178181

only one circle found with coordinates: (52.3675, 4.8985, 9.598958, 1279.8610666666666, 0.20088856799037302)
intersecion: [(52.3675, 23.727998130194806), (63.864687577361806, 4.898500000000001), (52.3675, -13.930998130194803), (40.87031242263819, 4.898499999999997)]
centroid: (52.3675, 4.8985)
ip_src: 145.220.0.55 dst (measured): 0.0 dst (calculated): 466.6206202376259
ip_

available measurements for ip: 10
NUMBER OF MEASUREMENT RESULTS FOR TARGET IP: 145.220.0.55 : 10
vp_anchor = 199.10.66.231 with results: 1.460768
vp_anchor = 213.225.160.239 with results: 9.67181
vp_anchor = 194.150.191.46 with results: 10.080243
vp_anchor = 192.65.184.54 with results: 17.059611
vp_anchor = 193.171.255.2 with results: 18.033813
vp_anchor = 5.28.0.17 with results: 20.369631
vp_anchor = 185.42.136.158 with results: 21.335698
vp_anchor = 76.26.115.194 with results: 93.358101
vp_anchor = 197.80.104.36 with results: 179.247495
vp_anchor = 200.7.84.24 with results: 179.891801

only one circle found with coordinates: (52.3485, 4.8315, 1.460768, 194.76906666666667, 0.03057119238214828)
intersecion: [(52.3485, 7.695738463973143), (54.09814029460361, 4.8315), (52.3485, 1.9672615360268573), (50.59885970539639, 4.831499999999999)]
centroid: (52.3485, 4.8315)
ip_src: 199.10.66.231 dst (measured): 0.0 dst (calculated): 5.013373300223824
ip_src: 213.225.160.239 dst (measured): 466.80

available measurements for ip: 10
NUMBER OF MEASUREMENT RESULTS FOR TARGET IP: 5.28.0.17 : 10
vp_anchor = 193.171.255.2 with results: 4.490919
vp_anchor = 194.150.191.46 with results: 19.152802
vp_anchor = 199.10.66.231 with results: 20.278647
vp_anchor = 145.220.0.55 with results: 20.37761
vp_anchor = 192.65.184.54 with results: 24.168527
vp_anchor = 185.42.136.158 with results: 28.595969
vp_anchor = 213.225.160.239 with results: 28.664983
vp_anchor = 76.26.115.194 with results: 136.11143
vp_anchor = 197.80.104.36 with results: 200.601989
vp_anchor = 200.7.84.24 with results: 255.824354

only one circle found with coordinates: (48.2085, 16.3695, 4.490919, 598.7891999999999, 0.09398668968764715)
intersecion: [(48.2085, 24.440986172431757), (53.58751490325701, 16.3695), (48.2085, 8.298013827568242), (42.82948509674299, 16.3695)]
centroid: (48.2085, 16.3695)
ip_src: 193.171.255.2 dst (measured): 0.0 dst (calculated): 213.94823034118167
ip_src: 194.150.191.46 dst (measured): 413.989388829

available measurements for ip: 20
NUMBER OF MEASUREMENT RESULTS FOR TARGET IP: 193.171.255.2 : 10
vp_anchor = 5.28.0.17 with results: 4.430461
vp_anchor = 5.28.0.17 with results: 4.898768
vp_anchor = 194.150.191.46 with results: 9.352871
vp_anchor = 194.150.191.46 with results: 9.373656
vp_anchor = 145.220.0.55 with results: 17.741474
vp_anchor = 145.220.0.55 with results: 17.832033
vp_anchor = 213.225.160.239 with results: 18.804128
vp_anchor = 213.225.160.239 with results: 18.881305
vp_anchor = 199.10.66.231 with results: 19.682417
vp_anchor = 199.10.66.231 with results: 19.810536

2 circles, case is not handled yet
intersecion: []
target:
52.3485 4.8315


available measurements for ip: 20
NUMBER OF MEASUREMENT RESULTS FOR TARGET IP: 192.65.184.54 : 10
vp_anchor = 194.150.191.46 with results: 13.058415
vp_anchor = 194.150.191.46 with results: 13.125371
vp_anchor = 199.10.66.231 with results: 15.456853
vp_anchor = 199.10.66.231 with results: 15.525966
vp_anchor = 145.220.0.55 with results: 16.975005
vp_anchor = 145.220.0.55 with results: 17.019012
vp_anchor = 213.225.160.239 with results: 18.951372
vp_anchor = 213.225.160.239 with results: 19.051271
vp_anchor = 193.171.255.2 with results: 20.506682
vp_anchor = 193.171.255.2 with results: 20.619004

2 circles, case is not handled yet
intersecion: []
target:
48.2085 16.3695


...

available measurements for ip: 10
NUMBER OF MEASUREMENT RESULTS FOR TARGET IP: 197.80.104.36 : 10
vp_anchor = 145.220.0.55 with results: 179.24711
vp_anchor = 199.10.66.231 with results: 179.268791
vp_anchor = 194.150.191.46 with results: 188.849492
vp_anchor = 213.225.160.239 with results: 189.225753
vp_anchor = 192.65.184.54 with results: 190.882659
vp_anchor = 193.171.255.2 with results: 198.735692
vp_anchor = 185.42.136.158 with results: 199.843395
vp_anchor = 5.28.0.17 with results: 200.201884
vp_anchor = 76.26.115.194 with results: 265.60156
vp_anchor = 200.7.84.24 with results: 351.371689

2 circles, case is not handled yet
intersecion: []
target:
-34.8805 -56.1805


available measurements for ip: 10
NUMBER OF MEASUREMENT RESULTS FOR TARGET IP: 200.7.84.24 : 10
vp_anchor = 76.26.115.194 with results: 156.571133
vp_anchor = 213.225.160.239 with results: 179.484846
vp_anchor = 145.220.0.55 with results: 179.592125
vp_anchor = 199.10.66.231 with results: 191.355974
vp_anchor = 192.65.184.54 with results: 206.403555
vp_anchor = 193.171.255.2 with results: 214.179098
vp_anchor = 194.150.191.46 with results: 223.897021
vp_anchor = 185.42.136.158 with results: 223.982992
vp_anchor = 5.28.0.17 with results: 256.96965
vp_anchor = 197.80.104.36 with results: 353.154232

ANYCAST??? (52.3675, 4.8985, 179.592125, 23945.616666666665) (39.9585, -75.6095, 156.571133, 20876.15106666667)
ANYCAST??? (39.9585, -75.6095, 156.571133, 20876.15106666667) (48.5795, 7.7485, 179.484846, 23931.312799999996)
intersecion: [(-27.504379182162857, -28.92561710587979), (-51.090342458324145, 63.56873455609446)]
centroid: (-39.2973608202435, 17.321558725107337)
ip_src: 76.26.115.194 

available measurements for ip: 20
NUMBER OF MEASUREMENT RESULTS FOR TARGET IP: 76.26.115.194 : 10
vp_anchor = 213.225.160.239 with results: 86.091113
vp_anchor = 213.225.160.239 with results: 86.094219
vp_anchor = 145.220.0.55 with results: 93.262403
vp_anchor = 145.220.0.55 with results: 93.360856
vp_anchor = 193.171.255.2 with results: 93.706048
vp_anchor = 193.171.255.2 with results: 93.749669
vp_anchor = 199.10.66.231 with results: 96.356084
vp_anchor = 199.10.66.231 with results: 96.469168
vp_anchor = 185.42.136.158 with results: 96.472689
vp_anchor = 185.42.136.158 with results: 96.481244

only one circle found with coordinates: (48.5795, 7.7485, 86.094219, 11479.2292, 1.8017939412964998)
intersecion: [(48.5795, 163.61723563606043), (151.69917040271108, 7.74850000000001), (48.57950000000002, -148.1202356360604), (-54.54017040271106, 7.7484999999999715)]
centroid: (48.57950000000001, 7.748499999999998)
ip_src: 213.225.160.239 dst (measured): 1.4185840236768777e-12 dst (calculated)

available measurements for ip: 10
NUMBER OF MEASUREMENT RESULTS FOR TARGET IP: 194.150.191.46 : 10
vp_anchor = 193.171.255.2 with results: 9.287943
vp_anchor = 145.220.0.55 with results: 9.949291
vp_anchor = 199.10.66.231 with results: 10.405232
vp_anchor = 192.65.184.54 with results: 13.162711
vp_anchor = 213.225.160.239 with results: 15.30287
vp_anchor = 5.28.0.17 with results: 18.96893
vp_anchor = 185.42.136.158 with results: 22.017569
vp_anchor = 76.26.115.194 with results: 99.105301
vp_anchor = 197.80.104.36 with results: 188.782544
vp_anchor = 200.7.84.24 with results: 224.321939

intersecion: [(57.96616809384082, 23.45018746356124), (40.438840441119815, 5.174104859128483), (58.876000829016206, 21.80736024242555)]
centroid: (52.42700312132562, 16.810550855038425)
ip_src: 193.171.255.2 dst (measured): 469.82258104736053 dst (calculated): 524.6190028218343
ip_src: 145.220.0.55 dst (measured): 806.8291781661466 dst (calculated): 524.6190028218343
ip_src: 199.10.66.231 dst (measured)

available measurements for ip: 10
NUMBER OF MEASUREMENT RESULTS FOR TARGET IP: 199.10.66.231 : 10
vp_anchor = 145.220.0.55 with results: 1.396323
vp_anchor = 194.150.191.46 with results: 10.479593
vp_anchor = 213.225.160.239 with results: 10.531173
vp_anchor = 192.65.184.54 with results: 15.536461
vp_anchor = 193.171.255.2 with results: 19.59411
vp_anchor = 5.28.0.17 with results: 20.07748
vp_anchor = 185.42.136.158 with results: 24.108938
vp_anchor = 76.26.115.194 with results: 96.393301
vp_anchor = 197.80.104.36 with results: 179.59141
vp_anchor = 200.7.84.24 with results: 191.756768

only one circle found with coordinates: (52.3675, 4.8985, 1.396323, 186.17639999999997, 0.029222476848218486)
intersecion: [(52.3675, 7.637553688707461), (54.039951056623494, 4.8985), (52.3675, 2.1594463112925393), (50.695048943376506, 4.898499999999999)]
centroid: (52.36750000000001, 4.8985)
ip_src: 145.220.0.55 dst (measured): 7.068789997788372e-13 dst (calculated): 5.013373300224122
ip_src: 194.150.1

In [9]:
geoloc_out_file = Path(".") / "../results/all_anchors_towards_all_anchors_geoloc.pickle"
with open(geoloc_out_file, "rb") as f:
    geoloc_results = pickle.load(f)


In [11]:
from geoloc_imc_2023.helpers import distance

for i, target_addr in enumerate(geoloc_results):
    if i > 15: break
    target_lat = anchors[target_addr]["latitude"]
    target_lon = anchors[target_addr]["longitude"]

    estimated_lat = geoloc_results[target_addr]["estimated_lat"]
    estimated_lon = geoloc_results[target_addr]["estimated_lon"]

    d_error = distance(target_lat, estimated_lat, target_lon, estimated_lon)
    print(f"geoloc results for target ip: {target_addr} distance error: {d_error}")

geoloc results for target ip: 145.220.0.55 distance error: 3.981388122601496
geoloc results for target ip: 5.28.0.17 distance error: 166.404396745
geoloc results for target ip: 193.171.255.2 distance error: 0.8226714520441889
geoloc results for target ip: 192.65.184.54 distance error: 3.5015730277578623
geoloc results for target ip: 197.80.104.36 distance error: 19.16556336974885
geoloc results for target ip: 200.7.84.24 distance error: 26.886725742565762
geoloc results for target ip: 76.26.115.194 distance error: 85.76798202864943
geoloc results for target ip: 194.150.191.46 distance error: 2.5171887386945353
geoloc results for target ip: 199.10.66.231 distance error: 6.835654928240526
